In [1]:
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D

## Plot colormap

In [2]:
def bargain(l1, m1, l2, m2):
    return (l1*m2 + l2*m1) / (l1*l2 + l1*m2 + l2*m1)

In [9]:
%matplotlib qt

# values range and lambda1
lambda1 = 10
minval = 0.01
maxval = lambda1 * 10
nb_points = 10
mu1 = np.linspace(minval, maxval, nb_points)
mu2 = np.linspace(minval, maxval, nb_points)
lambda2 = np.linspace(minval, maxval, nb_points)
mu1, mu2, lambda2 = np.meshgrid(mu1, mu2, lambda2)

# colors (=phis)
phi = list()
for i in range(mu1.shape[0]):
    for j in range(mu1.shape[1]):
        for k in range(mu1.shape[2]):
            phi.append(bargain(lambda1, mu1[i,j,k], lambda2[i,j,k], mu2[i,j,k])**10)

# create fig
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
plot = ax.scatter(mu1, mu2, lambda2, c=phi, alpha=1, cmap=plt.cm.hot_r)
ax.set_xlabel("Mu 1")
ax.set_ylabel("Mu 2")
ax.set_zlabel("Lambda 2")
fig.colorbar(plot, label="bargain")
plt.show()

## Formal calculations

In [1]:
from sympy import *

Create symbols : lambda, mu, nu (=lambda+mu), followers, leaders

In [2]:
users = {0,1}
lambd = { u: symbols('l'+str(u)) for u in users}
mu = { u: symbols('m'+str(u)) for u in users}
nu = { u: lambd[u]+mu[u] for u in users}
Leaders = { u:users.difference({u}) for u in users }
Followers = dict(Leaders)

Fill A and C matrices.

In [3]:
A = zeros(len(users), len(users))
C = zeros(len(users), len(users))

for j in users:
    C[j,j] = mu[j] / nu[j]
    for k in Leaders[j]:
        denominator = 0
        for l in Leaders[j]:
            denominator += nu[l]
        A[j,k] = mu[k] / denominator

Fill b and d vectors.

In [4]:
b = { u:zeros(len(users),1) for u in users }
d = { u:zeros(len(users),1) for u in users }

for i in users:
    d[i][i,0] = lambd[i] / nu[i]
    for j in Followers[i]:
        denominator = 0
        for l in Leaders[j]:
            denominator += nu[l]
        b[i][j,0] = lambd[i] / denominator

Solve p and q.

In [5]:
p = dict()
q = dict()
for i in users:
    p[i] = (eye(len(users)) - A).inv() * b[i]
    q[i] = C * (eye(len(users)) - A).inv() * b[i] + d[i]

Create bargain funct.

In [8]:
f = 0
for i in users:
    for j in Leaders[i]:
        f += q[i][j]
f = simplify(f)

Compute Hessian matrix.

In [16]:
variables = list(lambd.values()) + list(mu.values())
Hess = dense.hessian(bargain, variables).factor().simplify()

Check SPD.

In [36]:
# create x vector
x = list()
for i in range(Hess.shape[0]):
    x.append(Symbol('x'+str(i)))
x = Matrix(x)

# x_transpose dot Hess dot x
product = ((x.T).dot(Hess)).dot(x)

ShapeError: Matrix size mismatch: (4, 1) * (4, 4).

## With scipy.optimize

In [103]:
import numpy as np
from scipy.optimize import minimize, show_options

In [111]:
def bargain(x):
    l1, m1, l2, m2 = x[0], x[1], x[2], x[3]
    return - (l1*m2 + l2*m1) / (l1*l2 + l1*m2 + l2*m1)
alpha = 100
cons = ({'type':'ineq', 'fun': lambda x: alpha-x[0]-x[1]},
        {'type':'ineq', 'fun': lambda x: alpha-x[2]-x[3]})
bnds = ((0, None), (0, None), (0, None), (0, None))

In [112]:
res = minimize(bargain, (1, 100, 100, 1), method='SLSQP', bounds=bnds, constraints=cons)

In [113]:
res

     fun: -1.0
     jac: array([0.0100744, 0.       , 0.       , 0.       ])
 message: 'Optimization terminated successfully.'
    nfev: 30
     nit: 5
    njev: 5
  status: 0
 success: True
       x: array([1.11022302e-16, 9.92615989e+01, 9.92530816e+01, 2.53087962e-01])